In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install line-bot-sdk

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from flask import Flask, request, abort
 
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

# googlecolab上でngrokをできるなんか
from flask_ngrok import run_with_ngrok
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

torch.set_default_tensor_type('torch.cuda.FloatTensor') 
app = Flask(__name__)
run_with_ngrok(app)

line_bot_api = LineBotApi("TOKEN")    # チャネルアクセストークン
handler = WebhookHandler("SECRET")    # チャネルシークレット

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Tokenizerを設定
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
# tokenizer.do_lower_case = True

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained('/content/drive/My Drive/lyrics/rinnna_gpt2/output_re/')
model.to(device)
model.eval()
 
 
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
 
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
 
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
 
    return 'OK'
 
 
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.reply_token == "00000000000000000000000000000000":
        return

    # ラインで送られてきた文を入力とする
    PREFIX_TEXT = event.message.text
    input_text = tokenizer.encode(PREFIX_TEXT, return_tensors="pt") 

    output = model.generate(input_text, do_sample=True, max_length=300, num_return_sequences=1)

    # 出力する歌詞
    # 返信として返す歌詞
    lyrics = tokenizer.batch_decode(output)[0].replace("</s>", "").replace("<s>", "")
 
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=lyrics)
    )

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8826-34-134-9-148.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Jan/2022 02:15:40] "POST /callback HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
127.0.0.1 - - [28/Jan/2022 02:22:42] "POST /callback HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
127.0.0.1 - - [28/Jan/2022 02:35:06] "POST /callback HTTP/1.1" 200 -


In [ ]:
from flask import Flask, request, abort
 
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)
from flask_ngrok import run_with_ngrok


 
app = Flask(__name__)
run_with_ngrok(app)

line_bot_api = LineBotApi("TOKEN")    # チャネルアクセストークン
handler = WebhookHandler("SECRET")    # チャネルシークレット

 
 
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
 
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
 
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
 
    return 'OK'
 
 
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    # if event.reply_token == "00000000000000000000000000000000":
    #     return

 
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text)
    )

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7434-35-233-147-246.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Jan/2022 12:21:05] "POST / HTTP/1.1" 404 -
127.0.0.1 - - [27/Jan/2022 12:21:16] "POST / HTTP/1.1" 404 -


In [ ]:
from flask import Flask, request, abort
 
from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

from flask_ngrok import run_with_ngrok
 
app = Flask(__name__)
run_with_ngrok(app)
 
line_bot_api = LineBotApi("TOKEN")    # チャネルアクセストークン
handler = WebhookHandler("SECRET")    # チャネルシークレット
 
 
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
 
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
 
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
 
    return 'OK'
 
 
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    if event.reply_token == "00000000000000000000000000000000":
        return
 
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text)
    )

if __name__ == "__main__":
    app.run()   # ポート番号を8000に指定

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://be8f-104-199-136-127.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Jan/2022 13:59:47] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2022 13:59:52] "POST /callback HTTP/1.1" 200 -


In [ ]:

import torch
from transformers import T5Tokenizer, AutoModelForCausalLM
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
# tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained('/content/drive/My Drive/lyrics/rinnna_gpt2/output_re/')
model.to(device)
model.eval()

Downloading:   0%|          | 0.00/787k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [ ]:
PREFIX_TEXT = "神様、私は"
input_text = tokenizer.encode(PREFIX_TEXT, return_tensors="pt") 

In [ ]:
output = model.generate(input_text, do_sample=True, max_length=300, num_return_sequences=1)
print(tokenizer.batch_decode(output)[0].replace("</s>", "").replace("<s>", ""))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


神様、私は  (fu～n)チュッ!チュッ!チュチュチュチュッ!サマーパ～ティチュッ!チュッ!期待しちゃうわチュッ!朝早くお出かけねグループで青い海yes!my love!電車のって日帰り(fu～n)チュッ!チュッ!チュチュチュチュッ!サマーパ～ティチュッ!チュッ!期待しちゃうわチュッ!お天気に恵まれて海の家もいい雰囲気i love you!お昼ご飯バーベキュー片思いは切ないけれど今日はあの人と一緒!プリティな水着着て勝負勝負勝負パーティパーティ!夏の海はロマンチック!腕を組んでいいですか?ok!(fu～n)チュッ!チュッ!チュチュチュッ!サマーパ～ティチュッ!チュッ!期待しちゃうわチュッ!大好きなあの人とたまにしか目が合わないoh my god!他の娘には笑顔ね落ち込んでる場合じゃない積極的にバーベキュー料理の腕見せて!勝負勝負勝負パーティパーティ!夏の海はロマンチック!腕を組んでいいですか?ok!(fu～n)チュッ!チュチュチュチュッ!サマーパ～ティチュッ!チュッ!期待しちゃうわチュッ!大好きなあの人とたまにしか目が合わないoh my god!他の娘には笑顔ね落ち込んでる場合じゃない積極的にバーベキュー
